# Firefly

Firefly is a natural inspired metaheuristic search algorithm and its idea is that the fireflies (aka solutions) get attracted to the most luminous solution (aka fly) and try to follow that solution. Fireflies in natural don't get attracted to luminous objects but actualy get repulsed by lighting but for pedagogical reasons we assume that light attracts fireflies.

__FIRE-FLY__(_source_ , _destination_ , _num of iterations_ , _num of flies_ ) __returns__ a solution/route    
&emsp;generate _num of flies_ random solutions/routes between _source_ and _destination_  
&emsp;__for__ _num of iterations_ __do__  
&emsp;&emsp;__for__ i __of__ _num of flies_ __do__  
&emsp;&emsp;initialize _attractivness_ list for the other fireflies relative to the fly _i_  
&emsp;&emsp;&emsp;__for__ j __of__ _num of flies_ __do__  
&emsp;&emsp;&emsp;&emsp; __if__ i \= j __skip this iteration__ &emsp;&emsp;&emsp;&emsp;&emsp;// so we don't move fly in the direction of itself  
&emsp;&emsp;&emsp;&emsp; _attractivness_ \[j\] \= _luminosity_ of fly j relative to i  
&emsp;&emsp;&emsp;_move_ fly _i_ to fly _j_ which is the most luminous fly relative to it  
&emsp;&emsp;__return__ the most _luminous_ fly 

We will be doing shortest path problem as usual so our solution would be a route and _moving_ would be between fly/route to another fly/route, it would be just like doing crossover between the two lists/routes and substitute the moving route with the child of the crossover. __But__ the function of luminosity wouldn't be that the shorter of two paths would be more luminous and attractive.  

Navigation apps usually find the path to be travelled with algorithms like contractions hierarchy or graph neural networks which offers near exact solutions (or excat in the case of CH). But they learned that the best way to travel is not just the shortest path but althought the path that has fewer turns and the path the doesn't go into residence areas, read more in this article [your navigation app is making traffic unmanageable](https://spectrum.ieee.org/computing/hardware/your-navigation-app-is-making-traffic-unmanageable).

What can we do in our objective function so we don't always favor the shortest path without the need for massive datacenters. We can incorporate the value of how much the route doesn't have turns by counting the number of nodes in the route. If you go digging through `osmnx` source code and inspect how it parses osm data and filter them and remove unnecesary data to the topology of the graph generated, remember `simplify` option in [`osmnx.graph`](https://osmnx.readthedocs.io/en/stable/osmnx.html#module-osmnx.graph) module functions. you will find that library gives us nodes that changes the direction of a given road.  

_luminous_ function would be equal to the length of the route plus the number of nodes, because we are trying to minizing both of them and because the paths in our search space are relatively close to each other the addition of the number of nodes would make a difference.

We also need to have a notion of distance between two solutions/routes and that would be the number of common nodes between them.

In [ ]:
import osmnx as ox
from utilities import *
from itertools import filterfalse

In [ ]:
location_point = (43.661667, -79.395)
G = ox.graph_from_point(location_point, dist=300, clean_periphery=True, simplify=True)
graph_map = ox.plot_graph_folium(G, popup_attribute='name', edge_width=2)
fig, ax = ox.plot_graph(G)

In [ ]:
# marking both the source and destination node

nc = ['r' if node == 389677909 or node == 55808290 else '#336699' for node in G.nodes()]
ns = [50 if node == 389677909 or node == 55808290 else 8 for node in G.nodes()]
fig, ax = ox.plot_graph(G, node_size=ns, node_color=nc, node_zorder=2)

let's define the move function and luminosity function

In [ ]:
move = cross_over  # we want to merge two routes so cross_over function explained in GA notebook is enough

# the length of the route + number of curves and intersection of the route
def luminosity(G, route):
    return cost(G, route) + len(route)

# number of common nodes between two routes
def distance(route1, route2):
    return len(set(route1) & set(route2))

In [ ]:
num_of_iterations = 100
num_of_flies = 25
# initialize our starting population
flies = [randomized_search(G, 55808290, 389677909) for _ in range(num_of_iterations)]

fly _i_ perceives fly _j_ from distance we will have the final luminosity function to be the independent luminosity multiplied by factor dependet on the distance between the two flies, so $${luminosity}_{i} = {luminosity}_{j}*{e}^{-\gamma * distance}$$ and this gamma is the attractiveness coefficient between the flies which will be assumed constant to simplify the implementation.

In [ ]:
gamma = 2

## The Algorithm

In [ ]:
for _ in tqdm(range(num_of_iterations)):
    for i in range(num_of_flies):
        flies_luminosity = list() # for all flies except i
        for j in range(num_of_flies):
            if i == j: continue # skips getting luminosity of fly i
            flies_luminosity.append((j , luminosity(G, flies[j]))) # saving both the fly and its luminosity
        moving_fly = flies[i]
        # remember that the shortest path and the fewest number of nodes is more luminous for us
        # in this problem, hence using min function instead of max
        moving_to_fly = min(flies_luminosity, key = lambda fly : fly[1] * math.exp(-1 * gamma * distance(moving_fly, flies[fly[0]])))
        moving_to_fly = flies[moving_to_fly[0]]
        # updating
        flies[i] = move(moving_fly, moving_to_fly)

let's find out the most luminous fly

In [ ]:
route = min(flies, key = lambda fly : luminosity(G, fly))

In [ ]:
cost(G, route)

In [ ]:
fig, ax = ox.plot_graph_route(G, route)